In [ ]:
import os
import random
from tqdm import tqdm
import zipfile
from pycocotools.coco import COCO
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageDraw
from typing import List, Dict, Any, Optional
import matplotlib.pyplot as plt
from hydra import initialize, compose
import json
import gdown
from collections import defaultdict
import numpy as np
from omegaconf import OmegaConf

from data_manager import DatasetManager
from my_utils import verify_labels, draw_boxes

def main(cfg: Dict[str, Any]) -> None:
    dataset_cfg = cfg.dataset  # Access the nested dataset configuration
    
    # Initialize DatasetManager with the configuration
    dataset_manager = DatasetManager(dataset_cfg)
    img_ids = dataset_manager.get_img_ids(category_name='person', sample_size=cfg.sample_size)
    print(f"Sampled image IDs: {img_ids}")
    
    if not img_ids:
        print("No images available for sampling.")
        return

    dataset = dataset_manager.create_dataset(img_ids, transform=transforms.ToTensor())
    dataloader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)

    # Save labels in YOLO format
    dataset_manager.save_labels(img_ids)

    # Verify labels by displaying images with bounding boxes
    verify_labels(dataset_cfg)

    # Draw bounding boxes on samples and display them using matplotlib
    for idx, (images, targets) in enumerate(dataloader):
        if idx >= cfg.display_limit:
            break

        # Convert the tensor image to PIL image
        image = transforms.ToPILImage()(images[0])

        # Access the bounding boxes
        boxes = targets['boxes'].tolist()

        # Print debug information
        print(f"Image {idx + 1} has {len(boxes)} bounding boxes.")
        for box in boxes:
            print(f"Bounding box: {box}")

        # Draw the boxes on the image
        image_with_boxes = draw_boxes(image, boxes)

        # Display the image using matplotlib
        plt.imshow(image_with_boxes)
        plt.axis('off')
        plt.show()

if __name__ == "__main__":
    with initialize(config_path="config"):
        cfg = compose(config_name="config")
        print(OmegaConf.to_yaml(cfg))
        main(cfg)

/tmp/ipykernel_82823/2842199162.py:68: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="config"):


dataset:
  dataset:
    dataset_type: CrowdHuman
    files_to_download:
      CrowdHuman_train01.zip: https://drive.google.com/uc?id=134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y
      CrowdHuman_train02.zip: https://drive.google.com/uc?id=17evzPh7gc1JBNvnW1ENXLy5Kr4Q_Nnla
      CrowdHuman_train03.zip: https://drive.google.com/uc?id=1tdp0UCgxrqy1B6p8LkR-Iy0aIJ8l4fJW
      CrowdHuman_val.zip: https://drive.google.com/uc?id=18jFI789CoHTppQ7vmRSFEdnGaSQZ4YzO
      annotation_train.odgt: https://drive.google.com/uc?id=1UUTea5mYqvlUObsC1Z8CFldHJAtLtMX3
      annotation_val.odgt: https://drive.google.com/uc?id=10WIRwu8ju8GRLuCkZ_vT6hnNxs5ptwoL
    data_dir: ./crowdhuman
    annotations_dir: ./crowdhuman/annotations
    image_dir: ./crowdhuman/Images
    annotations_file: ./crowdhuman/annotation_train.odgt
sample_size: 50
batch_size: 1
num_workers: 4
display_limit: 10

Extracting CrowdHuman_train01.zip...
Extracting CrowdHuman_train02.zip...
